In [ ]:
from train import train_setup

In [2]:
trainer = train_setup(r'./train_config.yaml')

100%|██████████| 1000/1000 [00:00<00:00, 10619.06it/s]
Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing WrappedSegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing WrappedSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WrappedSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WrappedSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracke

In [3]:
# set(trainer.train_dataset.data_names).intersection(set(trainer.eval_dataset.data_names))

In [4]:
# trainer.train()

In [5]:
trainer.evaluate()

  0%|          | 0/150 [00:00<?, ?it/s]

{'eval_loss': 0.614561140537262,
 'eval_dice_group_0': 0.23280684649944305,
 'eval_dice_group_1': 0.6383125185966492,
 'eval_dice': 0.2436203509569168,
 'eval_runtime': 5.5812,
 'eval_samples_per_second': 26.876,
 'eval_steps_per_second': 26.876}

In [ ]:
print(trainer.train_dataset.dataset[0]['input'].shape)
trainer.train_dataset.dataset[0]['target'].shape

In [ ]:
print(trainer.eval_dataset[0]['input'].shape)
trainer.eval_dataset[0]['target'].shape

In [ ]:
trainer.train_dataset.dataset.augmentator()

In [ ]:
trainer.eval_dataset.augmentator() #.transforms_always

In [ ]:
trainer.model.loss.losses[0].weight

In [ ]:
from data_utils.dataset import load_datasets

In [ ]:
train_dataset, test_dataset = load_datasets(r'./train_config.yaml')

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

In [ ]:
set(test_dataset.data_images.keys()).intersection(set(train_dataset.data_images.keys()))

In [ ]:
for d in test_dataset:
    pass